In [25]:
import (
	"fmt"
	"log"
	"os"
	"time"

	"github.com/evertoncolling/poc-requests-go/pkg/api"
	"github.com/evertoncolling/poc-requests-go/pkg/dto"

    "github.com/janpfeifer/gonb/gonbui/plotly"
	grob "github.com/MetalBlueberry/go-plotly/generated/v2.34.0/graph_objects"
    ptypes "github.com/MetalBlueberry/go-plotly/pkg/types"
	"github.com/joho/godotenv"

    "github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

// Find a unit by external ID
func findUnitByExternalId(unitList *dto.UnitList, externalId string) (*dto.Unit, error) {
	for _, unit := range unitList.Items {
		if unit.ExternalId == externalId {
			return &unit, nil
		}
	}
	return nil, fmt.Errorf("unit not found with ExternalId: %s", externalId)
}

func getClient() api.CogniteClient {
    // Load environment variables
    err := godotenv.Load(".env")
    if err != nil {
        log.Fatalf("Error loading .env file")
    }

    // Read client credentials from environment variables
    clientID := os.Getenv("CLIENT_ID")
    clientSecret := os.Getenv("CLIENT_SECRET")
    tenantID := os.Getenv("TENANT_ID")
    cluster := os.Getenv("CDF_CLUSTER")
    project := os.Getenv("CDF_PROJECT")

    credentials := api.AzureADClientCredentials(
        clientID,
        clientSecret,
        tenantID,
        cluster,
    )
    clientConfig := api.ClientConfig{
        ClientName:  "poc-requests-go",
        Cluster:     cluster,
        Project:     project,
        Credentials: credentials,
    }
    client := api.NewCogniteClient(clientConfig)
    return client
}

// initialize client
var client = getClient()

%%
fmt.Println("### Testing fetching some time series")

// List time series
tsList, err := client.TimeSeries.List(100, false, "", "", nil, nil, "")
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Time Series Count:", len(tsList.Items))

### Testing fetching some time series
Time Series Count: 100


In [26]:
%%
// Filter time series
filter := &dto.TimeSeriesFilter{
	UnitQuantity: "Pressure",
}
filteredTsList, err := client.TimeSeries.Filter(filter, nil, 100, "", "", nil)
if err != nil {
	fmt.Println("Error:", err)
	return
}

fmt.Println("Filtered Time Series Count:", len(filteredTsList.Items))

Filtered Time Series Count: 100


In [27]:
func getUnitList() dto.UnitList {
	unitList, err := client.Units.List()
	if err != nil {
		fmt.Println("Error:", err)
		return dto.UnitList{}
	}
	return unitList
}
var unitList = getUnitList()

%%
fmt.Println("### Testing fetching the Unit catalog")
fmt.Println("Unit Count:", len(unitList.Items))

### Testing fetching the Unit catalog
Unit Count: 359


In [28]:
%%
// Fetch the latest data points
fmt.Println("### Latest Data Points")
externalIds := []string{"EVE-TI-FORNEBU-01-2", "EVE-TI-FORNEBU-01-3"}
var latestDataPointsQueryItems []dto.LatestDataPointsQueryItem
for _, externalId := range externalIds {
	latestDataPointsQueryItems = append(latestDataPointsQueryItems, dto.LatestDataPointsQueryItem{
		ExternalId: externalId,
	})
}
latestDataPoints, err := client.TimeSeries.RetrieveLatest(
	&latestDataPointsQueryItems,
	nil,
)
if err != nil {
	fmt.Println("Error:", err)
	return
}
for _, latestDataPoint := range latestDataPoints.Items {
	unit, err := findUnitByExternalId(&unitList, latestDataPoint.UnitExternalId)
	if err != nil {
		fmt.Println("Error:", err)
		return
	}
	fmt.Println(latestDataPoint.ExternalId+" ["+unit.Symbol+"]"+":", latestDataPoint.DatapointType)
}

### Latest Data Points
EVE-TI-FORNEBU-01-2 [°C]: &{datapoints:{timestamp:1745317200000  value:7.8}}
EVE-TI-FORNEBU-01-3 [°C]: &{datapoints:{timestamp:1745316903000  value:8.19}}


In [29]:
%%
// Get many data points (performance test)
fmt.Println("### Data Points (performance test)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId: "EVE-TI-FORNEBU-01-2",
		Start:      "300d-ago",
		End:        "now",
		Limit:      100000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}
dps := dataPoints.Items[0]
fmt.Println("Data Points External ID:", dps.ExternalId)
fmt.Println("Data Points Unit:", dps.UnitExternalId)
// not sure if there is a simpler way to get to the data points :)
switch v := dps.DatapointType.(type) {
case *dto.DataPointListItem_NumericDatapoints:
	fmt.Println("Data Points Count:", len(v.NumericDatapoints.Datapoints))
case *dto.DataPointListItem_StringDatapoints:
	fmt.Println("Data Points Count:", len(v.StringDatapoints.Datapoints))
case *dto.DataPointListItem_AggregateDatapoints:
	fmt.Println("Data Points Count:", len(v.AggregateDatapoints.Datapoints))
default:
	fmt.Println("Unknown data point type:", v)
}
fmt.Printf("Time taken: %s\n", elapsed)

### Data Points (performance test)
Data Points External ID: EVE-TI-FORNEBU-01-2
Data Points Unit: temperature:deg_c
Data Points Count: 63554
Time taken: 177.08775ms


In [30]:
%%
// let's try a more reasonable number of data points
fmt.Println("### Data Points (for plotting)")
items := []dto.DataPointsQueryItem{
	{
		ExternalId:  "EVE-TI-FORNEBU-01-2",
		Start:       "300d-ago",
		End:         "now",
		Aggregates:  []string{"average"},
		Granularity: "1h",
		Limit:       10000,
	},
}
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Prepare the data for plotting
dps := dataPoints.Items[0]
aggregateDatapointsType := dps.DatapointType.(*dto.DataPointListItem_AggregateDatapoints)
aggregateDatapoints := aggregateDatapointsType.AggregateDatapoints.Datapoints
values := make([]float64, 0, len(aggregateDatapoints))
timestamps := make([]time.Time, 0, len(aggregateDatapoints))
for _, datapoint := range aggregateDatapoints {
	if datapoint != nil {
		values = append(values, datapoint.Average)
		timestamp := time.Unix(0, datapoint.Timestamp*int64(time.Millisecond))
		timestamps = append(timestamps, timestamp)
	}
}
fmt.Printf("Time taken: %s\n", elapsed)
unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Plot the data
fig := &grob.Fig{
    Data: []ptypes.Trace{
        &grob.Scatter{
            X:    ptypes.DataArray(timestamps),
            Y:    ptypes.DataArray(values),
            Name: ptypes.S(dps.ExternalId),
            Line: &grob.ScatterLine{
                Color: ptypes.C("#6b9eff"),  // Light blue color for the line
                Width: ptypes.N(1),
            },
        },
    },
    Layout: &grob.Layout{
        Title: &grob.LayoutTitle{
            Text: ptypes.S(fmt.Sprintf("Fetched %d data points - hourly aggregates", len(values))),
            Font: &grob.LayoutTitleFont{
                Size:  ptypes.N(24.0),
                Color: ptypes.C("#ffffff"),  // White text
            },
        },
        PaperBgcolor: ptypes.C("rgba(0,0,0,0)"),  // Transparent background
        PlotBgcolor:  ptypes.C("rgba(0,0,0,0)"),  // Transparent plot area
        Font: &grob.LayoutFont{
            Family: ptypes.S("Helvetica"),
            Size:   ptypes.N(12),
            Color:  ptypes.C("#ffffff"),  // White text
        },
        Xaxis: &grob.LayoutXaxis{
            Showspikes:     ptypes.B(true),
            Spikemode:      grob.LayoutXaxisSpikemode("across"),
            Spikethickness: ptypes.N(1.0),
            Spikedash:      ptypes.S("solid"),
            Gridcolor:      ptypes.C("rgba(255,255,255,0.1)"),  // Subtle grid lines
            Linecolor:      ptypes.C("#ffffff"),                // White axis line
            Zerolinecolor:  ptypes.C("#ffffff"),                // White zero line
            Tickfont: &grob.LayoutXaxisTickfont{
                Color: ptypes.C("#ffffff"),  // White tick labels
            },
        },
        Yaxis: &grob.LayoutYaxis{
            Title: &grob.LayoutYaxisTitle{
                Text: ptypes.S(fmt.Sprintf("%s [%s]", unit.Quantity, unit.Symbol)),
                Font: &grob.LayoutYaxisTitleFont{
                    Color: ptypes.C("#ffffff"),  // White axis title
                },
            },
            Gridcolor:     ptypes.C("rgba(255,255,255,0.1)"),  // Subtle grid lines
            Linecolor:     ptypes.C("#ffffff"),                // White axis line
            Zerolinecolor: ptypes.C("#ffffff"),                // White zero line
            Tickfont: &grob.LayoutYaxisTickfont{
                Color: ptypes.C("#ffffff"),  // White tick labels
            },
        },
        Spikedistance: ptypes.I(-1),
        Legend: &grob.LayoutLegend{
            Orientation: grob.LayoutLegendOrientation("h"),
            Yanchor:     grob.LayoutLegendYanchor("bottom"),
            Y:           ptypes.N(1.02),
            Xanchor:     grob.LayoutLegendXanchor("right"),
            X:           ptypes.N(1.0),
            Font: &grob.LayoutLegendFont{
                Color: ptypes.C("#ffffff"),  // White legend text
            },
            Bgcolor: ptypes.C("rgba(0,0,0,0)"),  // Transparent legend background
        },
        Margin: &grob.LayoutMargin{
            L: ptypes.N(50),  // Left margin
            R: ptypes.N(30),  // Right margin
            T: ptypes.N(50),  // Top margin
            B: ptypes.N(50),  // Bottom margin
        },
        Hovermode:  ptypes.S("x"),
        Showlegend: ptypes.B(true),
        Height:     ptypes.N(600),
        Hoverlabel: &grob.LayoutHoverlabel{
            Bgcolor: ptypes.C("rgba(0,0,0,0.8)"),  // Semi-transparent black tooltip background
            Font: &grob.LayoutHoverlabelFont{
                Color: ptypes.C("#ffffff"),  // White tooltip text
            },
        },
    },
}

// Optional: Add hover template for better tooltip formatting
fig.Data[0].(*grob.Scatter).Hovertemplate = ptypes.ArrayOKValue(ptypes.S("%{x}<br>%{y:.2f} °C<extra></extra>"))

// Display the plot
plotly.DisplayFig(fig)

### Data Points (for plotting)
Time taken: 130.323875ms


In [31]:
// generateLineItems converts the values to ECharts line data points
func generateLineItems(values []float64) []opts.LineData {
	items := make([]opts.LineData, 0, len(values))
	for _, v := range values {
		items = append(items, opts.LineData{Value: v})
	}
	return items
}

%%
// Define the data points query
items := []dto.DataPointsQueryItem{
	{
		ExternalId:  "EVE-TI-FORNEBU-01-2",
		Start:       "300d-ago",
		End:         "now",
		Aggregates:  []string{"average"},
		Granularity: "1h",
		Limit:       10000,
	},
}

// Retrieve data points
start := time.Now()
dataPoints, err := client.TimeSeries.RetrieveData(
	&items,
	nil, nil, nil, nil, nil, nil, nil, nil,
)
elapsed := time.Since(start)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Prepare the data for plotting
dps := dataPoints.Items[0]
aggregateDatapointsType := dps.DatapointType.(*dto.DataPointListItem_AggregateDatapoints)
aggregateDatapoints := aggregateDatapointsType.AggregateDatapoints.Datapoints
values := make([]float64, 0, len(aggregateDatapoints))
timestamps := make([]time.Time, 0, len(aggregateDatapoints))
for _, datapoint := range aggregateDatapoints {
	if datapoint != nil {
		values = append(values, datapoint.Average)
		timestamp := time.Unix(0, datapoint.Timestamp*int64(time.Millisecond))
		timestamps = append(timestamps, timestamp)
	}
}
fmt.Printf("Time taken: %s\n", elapsed)

unit, err := findUnitByExternalId(&unitList, dps.UnitExternalId)
if err != nil {
	fmt.Println("Error:", err)
	return
}

// Convert timestamps to string format for the X-axis
xAxisData := make([]string, 0, len(timestamps))
for _, t := range timestamps {
	xAxisData = append(xAxisData, t.Format("2006-01-02 15:04"))
}

// Create a new line chart
line := charts.NewLine()

// Convert timestamps to formatted strings for x-axis
timeLabels := make([]string, len(timestamps))
for i, ts := range timestamps {
    timeLabels[i] = ts.Format("2006-01-02 15:04")
}

// Convert values to opts.LineData
lineData := make([]opts.LineData, len(values))
for i, v := range values {
    lineData[i] = opts.LineData{Value: float64(int(v*100))/100}  // Round to 2 decimals
}

// Configure the chart
line.SetGlobalOptions(
    charts.WithTitleOpts(opts.Title{
        Title: fmt.Sprintf("Fetched %d data points - hourly aggregates", len(values)),
        Subtitle: dps.ExternalId,
        TitleStyle: &opts.TextStyle{
            Color: "#ffffff",
        },
    }),
    charts.WithTooltipOpts(opts.Tooltip{
        Show: opts.Bool(true),
        Trigger: "axis",
        Formatter: "{b0} <br/> {a0}: {c0}°C",
    }),
    charts.WithGridOpts(opts.Grid{
        Left: "3%",
        Right: "5%",
        Top: "10%",
        Bottom: "10%",
        ContainLabel: opts.Bool(true),
    }),
    charts.WithLegendOpts(opts.Legend{
        Show: opts.Bool(true),
        Top: "3%",
        Right: "5%",
        Orient: "horizontal",
        TextStyle: &opts.TextStyle{
            Color: "#ffffff",
        },
    }),
    charts.WithYAxisOpts(opts.YAxis{
        Name: fmt.Sprintf("%s [%s]", unit.Quantity, unit.Symbol),
        NameLocation: "middle",
        NameGap: 50,
        Type: "value",
        AxisLabel: &opts.AxisLabel{
            Color: "#ffffff",
            Formatter: "{value}", // Corrected axis label format
        },
        AxisLine: &opts.AxisLine{
            LineStyle: &opts.LineStyle{
                Color: "#ffffff",
            },
        },
    }),
    charts.WithXAxisOpts(opts.XAxis{
        Name: "Time",
        AxisLabel: &opts.AxisLabel{
            Color: "#ffffff",
        },
        AxisLine: &opts.AxisLine{
            LineStyle: &opts.LineStyle{
                Color: "#ffffff",
            },
        },
    }),
    charts.WithInitializationOpts(opts.Initialization{
        Width: "1200px",
        Height: "600px",
        BackgroundColor: "transparent",
        Theme: "dark",
    }),
)

// Modify the series to use white color for the line
line.SetXAxis(timeLabels).
    AddSeries(dps.ExternalId,
        lineData,
        charts.WithLineChartOpts(opts.LineChart{
            Smooth: opts.Bool(true),
        }),
        charts.WithLineStyleOpts(opts.LineStyle{
            Color: "#6b9eff",  // Light blue line color
        }),
    )

// If you're using gonb_echarts:
err = gonb_echarts.Display(line, "width: 1200px; height: 800px; background: transparent;")
if err != nil {
    fmt.Printf("Error displaying chart: %v\n", err)
}

Time taken: 129.758958ms


In [32]:
%%
// Fetch data models
fmt.Println("\n### Testing fetching some data models")
dataModelsList, err := client.DataModeling.ListDataModels(
	1000, nil, nil, false, true,
)
if err != nil {
	fmt.Println("Error:", err)
	return
}

for _, dataModel := range dataModelsList.Items {
	fmt.Printf("Space: %s, External ID: %s, Version: %s\n", dataModel.Space, dataModel.ExternalId, dataModel.Version)
}
fmt.Println("Data Models Count:", len(dataModelsList.Items))


### Testing fetching some data models
Space: ChartsFDM, External ID: Sensors, Version: 1
Space: Geography, External ID: World_Cities, Version: 1
Space: PowerBI, External ID: TimeSeriesPowerBI, Version: 1
Space: PumpsDemo1, External ID: PumpsDemo, Version: 1
Space: PumpsDemo, External ID: Pumps, Version: 1
Space: RefineryData, External ID: RefineryData, Version: 1
Space: SLB-test3, External ID: SLB_test3, Version: 1
Space: WeatherData, External ID: Weather_Data, Version: 1
Space: cdf_360_image_schema, External ID: Image360, Version: v1
Space: cdf_3d_schema, External ID: Cdf3dGlobalTypes, Version: 1
Space: cdf_apm, External ID: ApmAppData, Version: v10
Space: cdf_apps_shared, External ID: SharedCogniteApps, Version: v1
Space: cdf_cdm, External ID: CogniteCore, Version: v1
Space: cdf_extraction_extensions, External ID: CogniteExtractorExtensions, Version: v1
Space: cdf_idm, External ID: CogniteProcessIndustries, Version: v1
Space: cdf_industrial_canvas, External ID: IndustrialCanvas, Ver

In [34]:
%%
// Search for CogniteTimeSeries instances
fmt.Println("\n### Testing searching for CogniteTimeSeries instances")
properties := []string{"name", "description"}
nodeList, err := client.DataModeling.InstancesSearch(
	dto.ViewReference{
		Type:       "view",
		Space:      "cdf_cdm",
		ExternalId: "CogniteTimeSeries",
		Version:    "v1",
	},
	"",
	nil,
	&properties,
	nil,
	nil,
	nil,
	10,
)
if err != nil {
	fmt.Println("Error:", err)
	return
}
fmt.Println("Node List Count:", len(nodeList.Items))
fmt.Println()

for _, node := range nodeList.Items {
	fmt.Printf("Space: %s, External ID: %s\n", node.Space, node.ExternalId)

	// Navigate through the nested properties structure
	for _, spaceValue := range node.Properties {
		// Cast the space value to a map
		if spaceMap, ok := spaceValue.(map[string]interface{}); ok {
			// Iterate through the view keys
			for viewKey, viewProperties := range spaceMap {
				fmt.Printf("View: %s\n", viewKey)

				// Cast the view properties to a map
				if propertiesMap, ok := viewProperties.(map[string]interface{}); ok {
					// Now print each property on a new line with the desired format
					for propKey, propValue := range propertiesMap {
						fmt.Printf("- %s: %v\n", propKey, propValue)
					}
				}
			}
		}
	}
	// Fetch the latest data points for the CogniteTimeSeries instance before now
	instanceId := dto.InstanceId{
		Space:      node.Space,
		ExternalId: node.ExternalId,
	}
	var latestDataPointsQueryItems []dto.LatestDataPointsQueryItem
	latestDataPointsQueryItems = append(latestDataPointsQueryItems, dto.LatestDataPointsQueryItem{
		InstanceId: &instanceId,
		Before:     "now",
	})
	latestDataPoints, err := client.TimeSeries.RetrieveLatest(
		&latestDataPointsQueryItems,
		nil,
	)
	if err != nil {
		fmt.Println("Error:", err)
		return
	}
	for _, latestDataPoint := range latestDataPoints.Items {
		// need to update protobuf to include the instance id
		fmt.Println("- latest data point:", latestDataPoint.DatapointType)
	}

	fmt.Println()
}


### Testing searching for CogniteTimeSeries instances
Node List Count: 10

Space: cdm-test, External ID: EVE-TI-5443
View: CogniteTimeSeries/v1
- assets: []
- isStep: false
- equipment: []
- sourceUnit: C
- name: TI-5443
- sourceCreatedTime: 2024-09-23T13:08:18.893+00:00
- type: numeric
- unit: map[externalId:temperature:deg_c space:cdf_cdm_units]
- latest data point: &{datapoints:{timestamp:1735689600000  value:81.62266062917679}}

Space: cdm-test, External ID: EVE-TI-5444
View: CogniteTimeSeries/v1
- type: numeric
- unit: map[externalId:temperature:deg_c space:cdf_cdm_units]
- assets: []
- isStep: false
- equipment: []
- sourceUnit: C
- name: TI-5444
- sourceCreatedTime: 2024-11-16T18:04:14.652+00:00
- latest data point: &{datapoints:{timestamp:1514937600000  value:3200}}

Space: paper_mill, External ID: TS_30_LT_101_PV
View: CogniteTimeSeries/v1
- isStep: false
- name: 30-LT-101-PV
- sourceId: PV_01
- type: numeric
- unit: map[externalId:dimensionless_ratio:percent space:cdf_cdm_un

In [35]:
%%
// GraphQL Query Example
fmt.Println("\n=== GraphQL Query Example ===")
graphQLQuery := `query MyQuery($nItems: Int) {
listCogniteUnit(filter: {space: {eq: "cdf_cdm_units"}}, first: $nItems) {
items {
  tags
  symbol
  space
  source
  sourceReference
  quantity
  name
  lastUpdatedTime
  externalId
  description
  createdTime
  aliases
}
}
}`

variables := map[string]interface{}{
	"nItems": 10,
}

graphQLResponse, err := client.DataModeling.GraphQLQuery(
	"cdf_cdm",
	"CogniteCore",
	"v1",
	graphQLQuery,
	variables,
)
if err != nil {
	fmt.Println("GraphQL query error:", err)
	return
}

// Check for GraphQL errors
if len(graphQLResponse.Errors) > 0 {
	fmt.Println("GraphQL errors:")
	for _, gqlErr := range graphQLResponse.Errors {
		fmt.Printf("  - %s\n", gqlErr.Message)
	}
	return
}

// Print the response as JSON
fmt.Println("GraphQL query successful!")
fmt.Println("Response data:")
jsonData, err := json.MarshalIndent(graphQLResponse.Data, "", "  ")
if err != nil {
	fmt.Println("Error marshaling response to JSON:", err)
	return
}
fmt.Println(string(jsonData))


=== GraphQL Query Example ===
GraphQL query successful!
Response data:
{
  "listCogniteUnit": {
    "items": [
      {
        "aliases": null,
        "createdTime": "2024-08-23T13:50:07.117Z",
        "description": "Foot per Square Second",
        "externalId": "acceleration:ft-per-sec2",
        "lastUpdatedTime": "2024-08-23T13:50:07.117Z",
        "name": "FT-PER-SEC2",
        "quantity": "Acceleration",
        "source": "qudt.org",
        "sourceReference": "https://qudt.org/vocab/unit/FT-PER-SEC2",
        "space": "cdf_cdm_units",
        "symbol": "ft/s²",
        "tags": null
      },
      {
        "aliases": null,
        "createdTime": "2024-08-23T13:50:07.117Z",
        "description": "Inch per Square second",
        "externalId": "acceleration:in-per-sec2",
        "lastUpdatedTime": "2024-08-23T13:50:07.117Z",
        "name": "IN-PER-SEC2",
        "quantity": "Acceleration",
        "source": "qudt.org",
        "sourceReference": "https://qudt.org/vocab/unit/I